In [2]:
import requests
import json
import pandas
import datetime

In [2]:
Exchange_Info_API = 'https://api.binance.com/api/v3/exchangeInfo'
Price_Info_API = 'https://api.binance.com/api/v3/klines'

In [3]:
def symbols_info():
    data = json.loads(requests.get(Exchange_Info_API).text)
    cols = ['symbol','status','baseAsset','quoteAsset']
    data = [[i[j] for j in cols] for  i in data['symbols']]
    return pandas.DataFrame(data,columns=cols) 

In [4]:
def Date_to_timestamp(_time,_format = '%d/%m/%Y %H:%M:%S'):
    return int(datetime.datetime.timestamp(datetime.datetime.strptime(_time,_format ))*1000)

In [5]:
def Price_info(symbol,step='1m',start=None,end=None,limit=500, base = '',quote=''):

    if start != None and type(start) == str:
        start = Date_to_timestamp(start)
    if end != None and type(end) == str:
        end = Date_to_timestamp(end)
    

    par = {
        'symbol':symbol,
        'startTime':start,
        'endTime':end,
        'interval':step,
        'limit':limit}


    cols =[
        'Kline open time',   #if a range of time
        'Open price',       #First trade price in the time range
        'High price',       #max buy in time
        'Low price',        #Min sell in time
        'Close price',      #Last trade price in the time range
        'Volume',           #Sum in time
        'Kline Close time',
        'Quote asset volume',   #Sum in time
        'Number of trades',     #Sum in time
        'Taker buy base asset volume',  #Sum in time
        'Taker buy quote asset volume', #Sum in time
        'Unused field, ignore'] # not use 
    p=pandas.DataFrame(json.loads(requests.get(Price_Info_API,params=par).text),columns=cols)
    p.insert(0,'quote', quote)
    p.insert(0,'base', base)
    p.insert(0,'Symbol', symbol)

    # base/quote = price
    return p

In [6]:
symbols = symbols_info()

data = pandas.concat(
    [Price_info(
        i['symbol'],
        '30m',
        '26/11/2022 00:00:00',
        '14/12/2022 00:00:00',
        1000,
        i['baseAsset'],
        i['quoteAsset']
        ) for _,i in symbols[symbols['status']=='TRADING'].iterrows()
    ],
    ignore_index=True)

In [8]:
symbols.to_csv("symbols.csv")
data.to_csv("data.csv")


In [39]:
headers = {
    'X-CMC_PRO_API_KEY': 'f7d541f8-896c-4695-89ce-c8908db48c5f'
}

params = {
    'limit': 5000
}

raw_symbols = json.loads(requests.get('https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest', headers=headers, params=params).text)

In [40]:
def get_symbol_attr(symbol):
    symbol_name = symbol['symbol']
    price = symbol['quote']['USD']['price']
    percent_change_7d = symbol['quote']['USD']['percent_change_7d']
    tags = ",".join(symbol['tags'])
    date_added = symbol['date_added']
    return [symbol_name, price, percent_change_7d, tags, date_added]
    
symbols = list(map(get_symbol_attr, raw_symbols['data']))

In [41]:

symbols

df = pandas.DataFrame(symbols, columns=columns)

In [42]:
df.to_csv("../datastore/symbols.csv")

In [43]:
df

,symbol,price,percent_change_7d,tags,date_added
0,BTC,1.684295e+04,0.790627,"mineable,pow,sha-256,store-of-value,state-chan...",2013-04-28T00:00:00.000Z
1,ETH,1.216737e+03,2.818052,"pos,smart-contracts,ethereum-ecosystem,coinbas...",2015-08-07T00:00:00.000Z
2,USDT,9.999156e-01,-0.017551,"payments,stablecoin,asset-backed-stablecoin,av...",2015-02-25T00:00:00.000Z
3,USDC,1.000030e+00,0.011159,"medium-of-exchange,stablecoin,asset-backed-sta...",2018-10-08T00:00:00.000Z
4,BNB,2.429735e+02,-2.004314,"marketplace,centralized-exchange,payments,smar...",2017-07-25T00:00:00.000Z
...,...,...,...,...,...
4995,WIS,1.910165e-02,26.944880,,2020-11-17T00:00:00.000Z
4996,BKK,1.382793e-01,-1.865411,"centralized-exchange,algorand-ecosystem",2020-02-12T00:00:00.000Z
4997,ZRO,4.128775e-06,8.816128,,2022-12-23T08:50:45.000Z
4998,EOSBULL,1.999831e-07,99.964899,,2020-04-06T00:00:00.000Z
